# Introduction

In this assignment you will be given a series of tasks about using the library `power-grid-model`. The agenda includes:

* Load input (From Power flow assignment)
* Numpy tips and PGM related recommendations
* Types of Dataset
* Miscellaneous PGM features
* Selective output
* Complex Simulations
    * Multiple Timeseries
    * Side topic: PGM DS library
    * Contingency Analysis
        * Topology caching
        * Simulation design exercise
    * Dynamic operating Envelope
        * Simulation design exercise
    * Additional content
        * Simulation design exercise

Throughout this notebook there are big notebook cells of code. 
To have a deeper understanding of the simulations, it is recommended to comment it all and uncomment line by line along with confirming the debugging print statements.

# Preparation

First import everything we need for this workshop:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from power_grid_model import (
    PowerGridModel,
    CalculationType,
    CalculationMethod,
    ComponentType,
    DatasetType,
    initialize_array,
    attribute_dtype,
    attribute_empty_value,
    power_grid_meta_data
)

from power_grid_model.validation import assert_valid_input_data, assert_valid_batch_data

from power_grid_model_ds import PowerGridModelInterface
from power_grid_model_ds.enums import NodeType
from power_grid_model_ds.visualizer import visualize

from utils import Timer
import psutil
import os
from collections import Counter

In [ ]:
np.set_printoptions(precision=3, linewidth=100)

# Topology Data and Timeseries load profiles 

(Reused from Power flow assignment)

In [ ]:
def load_input_data() -> dict[str, np.ndarray]:
    input_data = {}
    for component in [
        ComponentType.node,
        ComponentType.line,
        ComponentType.source,
        ComponentType.sym_load,
    ]:
        # Use pandas to read CSV data
        df = pd.read_csv(f"../data/{component.value}.csv")

        # Initialize array
        input_data[component] = initialize_array(DatasetType.input, component, len(df))

        # Fill the attributes
        for attr, values in df.items():
            input_data[component][attr] = values

        # Print some debug info
        print(f"{component:9s}: {len(input_data[component]):4d}")

    return input_data


input_data = load_input_data()


Create a PowerGridModel instance

In [ ]:
model = PowerGridModel(input_data=input_data)

Lets use these number timestamps for all calculations in the notebook.

In [ ]:
n_timestamps = 24 * 7

We use same timeseries profile as in the Power Flow assignment

In [ ]:
# Generate random load profile oh hourly data
dti = pd.date_range("2022-01-01", periods=n_timestamps, freq="h")
n_loads = len(input_data[ComponentType.sym_load])
load_id = input_data[ComponentType.sym_load]["id"]
load_p = input_data[ComponentType.sym_load]["p_specified"]
profile = np.tile(load_p, (n_timestamps, 1)) + 1e5 * np.random.randn(
    n_timestamps, n_loads
)

In [ ]:
# Initialize an empty load profile
load_profile_row_based = initialize_array(
    DatasetType.update, ComponentType.sym_load, profile.shape
)

load_profile_row_based["id"] = load_id
load_profile_row_based["p_specified"] = profile
load_profile_row_based["q_specified"] = 0.0

# Construct the update data
update_data_row_based = {ComponentType.sym_load: load_profile_row_based}


# Side topics related to numpy


## Calcualting Memory usage

In [ ]:
def print_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    print(f"Current memory usage: {mem_info.rss / 1024 / 1024:.2f} MB")


def print_memory_used_by_object(object):
    print(f"Memory used by object: {object.__sizeof__() / 1024 / 1024} MB")


def print_memory_used_by_dict(dictionary):
    print(
        f"Memory used by dictionary: {sum(object.__sizeof__() / 1024 / 1024 for object in dictionary.values())} MB"
    )


def print_memory_used_by_array(array: np.ndarray):
    print(f"Memory used by numpy array: {array.nbytes / 1024 / 1024} MB")


print_memory_usage()

print_memory_used_by_object([1] * 1000)
print_memory_used_by_object(
    {"a": [1] * 1000}
)  # This is because dictionary stores only references

print_memory_used_by_dict({"a": [1] * 1000})

print_memory_used_by_array(np.random.rand(1000, 1000))

## Correct dtype for PGM arrays


Use of initialize_array is recommended for most uses.
The reasoning for this recommendation should be clear by the end of this workshop.

However if you wish to translate the dtype into the one PGM uses, you can use  `initialize_array(dataset_type, component_type, empty_shape, empty=True)` or `dtype = power_grid_meta_data[dataset_type][component_type].dtype` to find the dtype of the row based data. You can use this to change to the appropriate type which is required by PGM.

In [ ]:
initialize_array(DatasetType.input, ComponentType.sym_load, 0, empty=True)
# power_grid_meta_data[DatasetType.input][ComponentType.sym_load].dtype

For dtype of individual attribute which would be required in columnar data creation, you can use `attribute_dtype(data_type, component_type, attribute)`. The missing value can be filled by `attribute_empty_value(data_type, component_type, attribute)` 

Tip: Check if a type-cast to this dtype creates a copy. Maybe such a copy is redundant.

In [ ]:
attribute_dtype(DatasetType.input, ComponentType.transformer, "sn")

All available attributes are visible via following:

In [ ]:
initialize_array(DatasetType.input, ComponentType.sym_load, 0, empty=True).dtype.names

## Views, copies and Memory location

A view is a windowed look into an array. Creating views is very inexpensive. 
A copy here is meant by an array which owns the data within it.

Check numpy documentation for when certain functions or features create a view or copy. 

* Views simplify assignment of values to arrays.
* Uneccesary copies here are the first you can aim to eliminate to improve overall performance.
* Beware of unintentional memory usage because of views.


https://numpy.org/doc/stable/user/basics.copies.html



* Checking if view or copy: if `array.base is None` -> copy; else -> view
* Finding out view belongs to which original array: if `array.base is original_array`

In [ ]:

x = np.arange(9)
y = x.reshape(3, 3)
assert y.base is not None  # .reshape() creates a view

z = y[[2, 1]]
assert z.base is None  # advanced indexing creates a copy

## Garbage collection for views/copies

### Holding base array


In [ ]:
def calculate_and_give_first_50_loading(model, update_data):
    output_data = model.calculate_power_flow(
        update_data=update_data,
        calculation_method=CalculationMethod.newton_raphson
    )
    return output_data[ComponentType.line]["loading"][:50]


first_50_loading = calculate_and_give_first_50_loading(model, update_data_row_based)


print(f"Shape of returned array: {first_50_loading.shape}")
print_memory_used_by_array(first_50_loading)

print(f"Actual shape of memory: {first_50_loading.base.shape}")
print_memory_used_by_array(first_50_loading.base)

Use .copy() to convert a view into a copy.

In [ ]:
def calculate_and_give_first_50_loading(model, update_data):
    output_data = model.calculate_power_flow(
        update_data=update_data,
        calculation_method=CalculationMethod.newton_raphson,
        threading=0
    )
    return output_data[ComponentType.line]["loading"][:50].copy()


first_50_loading = calculate_and_give_first_50_loading(model, update_data_row_based)
print(f"Shape of returned array: {first_50_loading.shape}")
print_memory_used_by_array(first_50_loading)
assert first_50_loading.base is None

## Data Contiguity

What is Contiguity in numpy arrays?
- Contiguity means all array elements are laid out sequentially in memory.
    - C contiguous: row major where rows are stored one after another. ie. All elements are in order of right to left dimension. Rightmost dimension is fast changing. (This is default in numpy and also required by PGM.)
    - F contiguous: column-major where columns stored one after another. ie. All elements are in order of left to right dimension. Leftmost dimension is fast changing.

The input and update dataset needs to be a C contigious dataset before PGM can make use of it. While PGM output is C contigious. PGM internally converts it to be C contigious if its not.

A basic sliced dataset would retain contiguity. Whereas a transposed or advanced sliced array would not.
A reshaped dataset mostly would be C contigious.

Check `arr.flags` for Contiguity.
This requirement for PGM and for further uses can be pre-planned.


In [ ]:
loads = np.zeros((1000, 10),dtype=np.float64)
print(f"1. {loads.flags}")
print(f"2. {loads.reshape(10, 20, 50).flags}")
print(f"3. {loads.T.flags}")
print(f"5. {np.transpose(loads.reshape(10, 20, 50), axes=(2, 0, 1)).flags}")

print(f"4. {np.ascontiguousarray(loads) is loads}")

`initialize_array` handles contiguity and dtype requirement for row based data without user having to think about it. 
For columnar data however the responsibility lies with the user.

Questions:
- Does a view own its data?
- Give reasoning for flags of 3.

## Accessing and assigning to PGM arrays

PGM component has layout of (scenarios, components in a scenario)

In [ ]:
loads = initialize_array(DatasetType.update, ComponentType.sym_load, (1000, 10))
loads[:] = (99, 1, 100.0, 100.0)

def copy_or_view(arr: np.ndarray) -> str:
    return "copy" if arr.base is None else "view"

In [ ]:
print("Single batch all components: `loads[:, 0]` gets a :", copy_or_view(loads[:, 0]))
print("All batch single components: `loads[0, :]` gets a :", copy_or_view(loads[0, :]))
print("Sliced batch all components: `loads[:5, :]` gets a :", copy_or_view(loads[:5, :]))
print("Multiple batch via advanced indexing all components: `loads[[1,2,3], :]` gets a :", copy_or_view(loads[[1,2,3], :]))  # Advanced indexing creates a copy
print("All batch Multiple components via advanced indexing: `loads[:, [1,2,3]]` gets a :", copy_or_view(loads[:, [1,2,3]]))  # Deceptive: view of another copy

### Attribute access

Attribute access has similar behavior

In [ ]:
print("Single batch all components: `loads['p_specified']` gets a :", copy_or_view(loads["p_specified"]))
print("Single batch all components: `loads['p_specified'][:,0]` gets a :", copy_or_view(loads["p_specified"][:, 0]))
print("Single batch all components: `loads['p_specified'][0, :]` gets a :", copy_or_view(loads["p_specified"][0, :]))
print("Sliced batch all components: `loads['p_specified'][:5, :]` gets a :", copy_or_view(loads["p_specified"][:5, :]))
print("Single batch all components: `loads['p_specified'][[1, 2, 3], :]` gets a :", copy_or_view(loads["p_specified"][[1, 2, 3], :]))
print("Single batch all components: `loads['p_specified'][:, [1, 2, 3]]` gets a :", copy_or_view(loads["p_specified"][:, [1, 2, 3]])) 

Its recommended to index with field first (`loads["p_specified"][...]` instead of `loads[...]["p_specified"]`) because advanced indexing on structured arrays can create copies.

Multi field indexing

In [ ]:
print("Multiple attributes: `loads[['p_specified', 'q_specified']][:, [1, 2, 3]]` gets a :", copy_or_view(loads[["p_specified", "q_specified"]]))

A common source of confusion is when dealing with numpy arrays view and copy variables. They look the same. 
* Modifications on view modifies the view as well as the original array.
* Modifications on copy modifies only that copy.


In [ ]:
loads = initialize_array(DatasetType.update, ComponentType.sym_load, (1000, 4))
loads[:] = (99, 1, 100.0, 100.0)

x = loads["p_specified"][:3, :]
loads["p_specified"][:3, :] = 123456
print(x)
print(x.base is loads)  # x is a view of loads


In [ ]:
loads = initialize_array(DatasetType.update, ComponentType.sym_load, (1000, 4))
loads[:] = (99, 1, 100.0, 100.0)

x = loads["p_specified"][[1, 2, 3], :]
loads["p_specified"][[1, 2, 3], :] = 123456
print(x)
print(x.base is loads)  # x is a copy of the loadings of scenario 1 2 3

#### Deceiptive: It was a view of a copy

In [ ]:
loads = initialize_array(DatasetType.update, ComponentType.sym_load, (1000, 4))
loads[:] = (99, 1, 100.0, 100.0)

x = loads["p_specified"][:, [1, 2, 3]]
loads["p_specified"][:, [1, 2, 3]] = 123456
print(x)
print(x.base is loads)

### Assignment to arrays

Assigning means setting values to the array.

#### Via tuple

In [ ]:
loads = initialize_array(DatasetType.input, ComponentType.asym_load, (10, 6))
loads["p_specified"] = 1000
loads["p_specified"] = [[1000], [1000], [1000], [1000], [1000], [1000]]
# fields are in this order: ('id', 'node', 'status', 'type', 'p_specified', 'q_specified')
loads[:, 0] = (1, 2, 1, 0, 123, 0)
loads[:] = (1, 2, 1, 0, 123, 0)
loads[:2]


Reshaping the destination arrays is also possible since a view gets created.
Its possible to assign values this way.

In [ ]:
loads = initialize_array(DatasetType.input, ComponentType.asym_load, (10, 6))
reshaped_loads = loads.reshape(2, 5, 6)
reshaped_loads[0, :, :] = (1, 2, 1, 0, 123, 0)
reshaped_loads[1, :, :] = (3, 4, 5, 0, 456, 0)
loads[[2, -2]]

Use functions with out parameter to write results directly into pre-allocated arrays. for eg. arrays created via initialize_array

# Types of datasets 

### Row based format

In [ ]:
print_memory_used_by_array(update_data_row_based[ComponentType.sym_load])

In [ ]:
# Run Newton Raphson power flow (this may take a minute...)
with Timer("Batch Calculation using Newton-Raphson row based data"):
    output_data = model.calculate_power_flow(
        update_data=update_data_row_based,
        calculation_method=CalculationMethod.newton_raphson,
        threading=0,
    )

print_memory_used_by_dict(output_data)

### Columnar data format

In [ ]:
# Construct the update data
update_data_columnar = {ComponentType.sym_load: {"p_specified": profile}}

print_memory_used_by_array(profile)  # in MB

PGM Columnar data is all attributes array are fed in indvidually as separate arrays.
This enables us to skip empty or redundant attribute data.
Skipping ids is also possible under certain conditions.

In [ ]:
with Timer("Batch Calculation using Newton-Raphson columnar data"):
    output_data = model.calculate_power_flow(
        update_data=update_data_columnar,
        calculation_method=CalculationMethod.newton_raphson,
        threading=0
    )

### Selective Output

It is possible to output only selected attributes via output_component_types. The effect on memory usage is huge.

In [ ]:
with Timer("Batch Calculation using Newton-Raphson columnar data"):
    output_data_selective = model.calculate_power_flow(
        update_data=update_data_columnar,
        calculation_method=CalculationMethod.newton_raphson,
        output_component_types={ComponentType.line: ["loading"]},
        threading=0
    )

print(
    f"output_data_selective contains following: \n"
    f"Keys: {output_data_selective.keys()}. \n"
    f"Attributes: {output_data_selective[ComponentType.line].keys()}"
)
print_memory_used_by_array(output_data_selective[ComponentType.line]["loading"])

Lets define all limit related attributes to use in upcoming simulation

In [ ]:
limit_related_output = {
    ComponentType.line: ["loading"],
    ComponentType.node: ["u_pu"],
    ComponentType.transformer: ["loading"],
}

#### Clear memory

In [ ]:
del load_profile_row_based, output_data, output_data_selective

# Miscellanious PGM features

## Sparse datasets

For unequal update sets use linear data + pointers. 
This way of providing update would likely be a niche.

Columnar sparse dataset representation for example can be as follows:

In [ ]:
update_data_sparse = {
    ComponentType.line: {
        "indptr": [0, 1, 1, 2],
        "data": {
            "p_specified": [1000, 2000]
        }
    }
}

## Validating large batch data

Since the validator for large batch dataset is expensive, we can choose to validate only some part of data.
The result of the validator is also difficult to interpret with many errors on a large dataset.
Maximum data errors typically occur in the input dataset.
Hence we can slice the update data to only certain batches and validate it.

In [ ]:
with Timer("Validation of full batch data"):
    assert_valid_batch_data(
        input_data=input_data,
        update_data=update_data_row_based,
        calculation_type=CalculationType.power_flow,
    )

with Timer("Validation of partial batch data"):
    assert_valid_batch_data(
        input_data=input_data,
        update_data={k: v[:5, :] for k, v in update_data_row_based.items()},
        calculation_type=CalculationType.power_flow,
    )

# Complex simulations

Often times we wish to run multiple simulations at once. 
These power flow calculations have multiple dimensions associated with them where the information regarding that particular simulation is put in.
The way to simulate these via PGM most effectively is to flatten all the excess dimension into one, ie. of batches. 
This makes the entire simulation including and most likely excluding PGM most efficient, readable and less bug-prone.


## Multiple Timeseries

We are now going to simulate multiple timeseries in a single power flow calculation and later the results would be aggregated from the output numpy arrays.


Here we have a EV growth related scenario in a neighbourhood.
For this simulation we have yearly EV consumption profiles available for 10 different customers.
We shall subset these profiles and timestamps to align with the data in this notebook  

In [ ]:
# Hourly EV profile data for one year. Row = hours, column = profiles
ev_profile = (
    pd.read_csv("../data/ev_profile.csv").to_numpy(
        dtype=np.float64, copy=False, na_value=np.nan
    )
    * 1000
)  # kw

# Select first n_timestamps and first n profiles
n_ev_profiles = 10

assert n_timestamps <= ev_profile.shape[0]
assert n_ev_profiles <= ev_profile.shape[1]
ev_profile = ev_profile[:n_timestamps, :n_ev_profiles] # Hint: Slicing
print(f"Shape of ev_profile: {ev_profile.shape}") # Prints: (n_timestamps * n_ev_profiles) 



Since the EV growth at each node is uncertain, we can sample different EV growth scenarios.
Lets consider: 
* The maximal EV sales in a region is a known quantity (denoted by n_vehicles).
* The spread of EVs across each loaded nodes is equal (denoted by pvals of n_vehicle_each_node).
* We conduct n_experiments samples for monte carlo simulation
* Each EV profile is assigned to each EV equally random (denoted by pvals of distributions).

In [ ]:
n_vehicles = 100000
n_experiments = 11

rng = np.random.default_rng(12345)

n_vehicle_each_node = rng.multinomial(
    n=..., pvals=[1 / ...] * ..., size=...
)
print(f"Shape of n_vehicle_each_node: {n_vehicle_each_node.shape}") # Prints: (n_experiments * n_loads) 

distributions = rng.multinomial(
    n=n_vehicle_each_node, pvals=[1 / ...] * ..., size=None
)
print(f"Shape of distributions: {distributions.shape}") # Prints: (n_experiments, n_loads, n_ev_profiles) 

ev_timeseries = np.transpose(... @ (...).T, axes=(0, 2, 1))
print(f"Shape of ev_timeseries: {ev_timeseries.shape}") # Prints: (n_experiments, n_timestamps, n_loads) 

ev_timeseries_q = ev_timeseries * ... # assume power factor of 0.95

# Hint: broadcasting (n_timestamps, n_loads) to (n_experiments, n_timestamps, n_loads) 
combined_timeseries = profile[...] + ev_timeseries 
print(f"Shape of combined_timeseries: {combined_timeseries.shape}") # Prints: (n_experiments, n_timestamps, n_loads) 


Create columnar update data and run calculation

In [ ]:
update_data_multi_timeseries = {
    ComponentType.sym_load: {
        "p_specified": combined_timeseries.reshape(..., ...), # All update data Should be 2D eventually
        "q_specified":  ev_timeseries.reshape(..., ...),
    }
}

In [ ]:
with Timer("Run multitimeseries PF"):
    output_data = model.calculate_power_flow(
        update_data=update_data_multi_timeseries,
        calculation_method=CalculationMethod.newton_raphson,
        output_component_types=limit_related_output,
        threading=0
    )

reshaped_loading_data = output_data[ComponentType.line]["loading"].reshape(..., ..., ...)
print(f"Shape of reshaped_loading_data: {reshaped_loading_data.shape}") # Prints: (n_experiments, n_timestamps, n_loads) 


Aggregate result on the basis of experiments of monte carlo simulation

In [ ]:
np.mean(reshaped_loading_data, axis=...) # Shape: (n_timestamps, n_loads) 

In [ ]:
del output_data, reshaped_loading_data

When PF is run in loop


In [ ]:
load_id_loop = np.tile(input_data[ComponentType.sym_load]["id"], (n_timestamps, 1))
output_datas = []
with Timer("Run multi-timeseries PF in loop"):
    for experiment in range(n_experiments):
        update_data_loop = {
            ComponentType.sym_load: {
                "id": load_id_loop,
                "p_specified": combined_timeseries[experiment, :, :],
                "q_specified": ev_timeseries_q[experiment, :, :],
            }
        }
        output_data = model.calculate_power_flow(
            update_data=update_data_loop,
            calculation_method=CalculationMethod.newton_raphson,
            output_component_types=limit_related_output,
            threading=0,
        )
        output_datas.append(output_datas)

del output_datas


The difference between this method and one big batch is indeed not very large, but this is a significant for performance improvement. 

The combining/seprating numpy arrays is a lot more wasteful.

The calculating of mean on `output_datas` list is more difficult than a numpy array.

Always keep the components within a single scenario as the rightmost dimension (aka. fast moving dimension). 
In this example it would be `n_loads`.
A transposing of array would likely be required otherwise which is an extra copy.

## PGM DS library

Converting to and from a PGM DS grid can be done via `PowerGridModelInterface`.

In [ ]:
core_interface = PowerGridModelInterface(input_data=input_data)
grid = core_interface.create_grid_from_input_data()
print(grid.line)

### Visualization of grid

In [ ]:
visualize(grid)

### Graph operations

Some graphical operations possible via PGM-DS. Custom ones can be implemented in your simulation via rustworkx.

In [ ]:

grid.node.node_type[0] = NodeType.SUBSTATION_NODE

# Downstream nodes from node 1 with the substation node as reference
print(f"1. {grid.get_downstream_nodes(node_id=1)}")

# In case of multiple substations, find nearest one
print(f"2. {grid.get_nearest_substation_node(node_id=10).id}")

# Find all paths between node 1 and node 10
print(f"3. {grid.graphs.active_graph.get_all_paths(ext_start_node_id=1, ext_end_node_id=10)}")

# Find shortest path between node 1 and node 10
print(f"4. {grid.graphs.active_graph.get_shortest_path(ext_start_node_id=1, ext_end_node_id=10)}")

# All components in this graph
print(f"5. {len(grid.graphs.active_graph.get_components())}")

# All components connected to node 1
print(f"6. {len(grid.graphs.active_graph.get_connected(node_id=1))}")

# All components connected to node 0, ignoring nodes 1 and 2
print(f"7. {len(grid.graphs.active_graph.get_connected(node_id=0, nodes_to_ignore=[1, 2]))}")


### Array operations

Many array operations are possible. Such operations are possible via numpy too but are not as intuitive.

In [ ]:
grid.line.filter(from_node=[0, 1, 2, 3, 4], to_node=[0, 1, 2, 3, 4, 5, 6, 7], mode_="AND")

### Extended arrays

Extra information can be stored along with arrays via a way described in https://power-grid-model-ds.readthedocs.io/en/stable/examples/model/grid_extensions_examples.html

Note that concatenating new fields to row based arrays would be inefficient as they would need to be reconstructed before using in PGM.

## Contingency Analysis with timeseries

Here we perform a selective contingency analysis on specific branches for all timesteps.

Use graph operations to find top 5 ranked branches which most nodes depend on.

(Assumption: Grid is radial and nodes are assigned with from_ndoe at source side.)

In [ ]:
# Lines with most most number of dependent nodes
n_contingencies = 5

all_branches = []
for node in grid.node.id:
    paths = grid.graphs.active_graph.get_all_paths(ext_start_node_id=..., ext_end_node_id=...)
    for path in paths:
        all_branches += [(path[idx], path[idx + 1]) for idx in range(len(path) - 1)]

print(f"Path list with length {len(all_branches)}")
print(all_branches)

counts = Counter(all_branches)
top_branches_from_to_counts = counts.most_common(n=...)
nodes_from = [i[0][0] for i in top_branches_from_to_counts]
nodes_to = [i[0][1] for i in top_branches_from_to_counts]

top_branches_id = grid.line.filter(from_node=..., to_node=..., mode_="AND").id

print(f"top {n_contingencies} branches {top_branches_id}")

Design batches for simulation in PGM

In [ ]:
combos_contingencies = 1 - np.eye(..., dtype=np.uint8)

line_update = initialize_array(
    DatasetType.update,
    ComponentType.line,
    (... * ..., ...),
)
line_update["id"] = top_branches_id
line_update_reshaped = line_update.reshape(n_contingencies, n_timestamps, n_contingencies)
line_update_reshaped["from_status"] = combos_contingencies[:, np.newaxis, :]
line_update_reshaped["to_status"] = combos_contingencies[:, np.newaxis, :]

Run contingency analysis on timeseries by using Timeseries data from earlier cell

In [ ]:
load_profile_combined = initialize_array(
    DatasetType.update,
    ComponentType.sym_load,
    (n_contingencies * n_timestamps, n_loads),
)

load_profile_combined["p_specified"] = np.broadcast_to(
    profile, (n_contingencies, n_timestamps, n_loads)
).reshape(-1, n_loads)
# Alternatively:
# load_profile_combined["p_specified"].reshape(n_contingencies, n_timestamps, n_loads)[:] = profile

load_profile_combined["q_specified"] = 0.0

update_data_combined = {
    ComponentType.line: line_update,
    ComponentType.sym_load: load_profile_combined,
}

with Timer("Batch Calculation with contingencies and load profiles"):
    output_data_combined = model.calculate_power_flow(
        update_data=update_data_combined,
        calculation_method=CalculationMethod.newton_raphson,
        output_component_types=limit_related_output,
        threading=0
    )

Plot loading on first line for all situations

In [ ]:
plt.plot(output_data_combined[ComponentType.line]["loading"][:, 0])

In [ ]:
del output_data_combined

#### Question regarding topology caching

Instead of designing the simulation like we did in this way:

```
1 0 0 0 timestep 1
1 0 0 0 timestep 2
1 0 0 0 timestep 3
...

0 1 0 0 timestep 1
0 1 0 0 timestep 2
0 1 0 0 timestep 3
...

```

What if contingencies are designed this way?


```
0 1 1 1   timestep 1
1 0 1 1
1 1 0 1
1 1 1 0

0 1 1 1   timestep 2
1 0 1 1
1 1 0 1
1 1 1 0
```


Another alternative to is to cache internal state of PowerGridModel object.

In [ ]:
model_topology_2 = model.copy()

### Simulation design exercise

* Can you give an example on how you can set up a network reconfiguration study based on the example above?
* What graph related operations are needed?
    * Is it possible to do those via PGM-DS?
* What result attributes are to be checked? Do they need to be aggregated?

## Dynamic operating envelope

Calculate the operating envelope for all timesteps at node 10 based on voltage limits.


Find the sym load connected to node 10

In [ ]:
sym_load_node_10 = grid.sym_load.filter(node=10).id
sym_load_node_10_idx = model.get_indexer(ids=sym_load_node_10, component_type=ComponentType.sym_load)[0]
print(sym_load_node_10_idx)

* Create a grid of P and Q values between a max and minimum range.
* Create multiple timeseries for each point in the grid
* Add the values in the grid to the load at node 10. Rest timeseries would remain the same.

In [ ]:
min_p = -1e7
max_p = 1e7
min_q = -1e7
max_q = 1e7
resolution = 11

p_linear_values = np.linspace(start=..., stop=..., num=...)
q_linear_values = np.linspace(start=..., stop=..., num=...)

p_values, q_values = np.meshgrid(p_linear_values, q_linear_values, copy=False)
print(f"p_values shape: {p_values.shape}") # Prints: (resolution, resolution)
print(f"q_values shape: {q_values.shape}") # Prints: (resolution, resolution)

p_envelope = np.tile(profile, (resolution, resolution, 1, 1))
print(f"p_envelope shape: {p_envelope.shape}") # Prints: (resolution, resolution, n_timestamps, n_loads)

p_envelope[:, :, :, sym_load_node_10_idx] += p_values[..., ..., ...]

q_envelope = np.zeros(p_envelope.shape)
q_envelope[:, :, :, sym_load_node_10_idx] += q_values[..., ..., ...]


Create update data and run simulation

In [ ]:
update_data_envelope = {
    ComponentType.sym_load: {
        "p_specified": p_envelope.reshape(..., ...),
        "q_specified": q_envelope.reshape(..., ...),
    }
}

In [ ]:
with Timer("Batch Calculation with grid of P and Q to calculate an envelope of operation"):
    output_data_envelope = model.calculate_power_flow(
        update_data=update_data_envelope,
        calculation_method=CalculationMethod.newton_raphson,
        output_component_types=limit_related_output,
        threading=0
    )

nodal_voltages = output_data_envelope[ComponentType.node]["u_pu"].reshape(..., ..., ..., ...)
print(f"nodal_voltages shape: {nodal_voltages.shape}")  # Prints: (resolution, resolution, n_timestamps, n_nodes)

Next Step to calculate the envelope:

* Calculate the index mask where the voltage does not cross above or below the voltage limit for each timestep
* Apply this mask on the grid of P and Q for each time step.

In [ ]:
u_low_lim = 0.95
u_high_lim = 1.03

max_voltage_scenario = np.max(nodal_voltages, axis=...) > ...
min_voltage_scenario = np.min(nodal_voltages, axis=...) < ...
points_outside_limits =  np.logical_or(max_voltage_scenario, min_voltage_scenario)


In [ ]:
plt.imshow(points_outside_limits[:,:, 0])

### Simulation Design Exercise

* Can you give anadditional examples of calculating flexibility envelopes? Maybe with more details? (based on technical limits and not economic ones) 
* What result attributes are to be checked? Do they need to be aggregated?

# Additional Content

#### Additional simulation design exercise

* Can you give an example of how we can implement a control related action via PGM? 
* Draw a block diagram of the control?
    * Is this scheme efficient?
    * How do we create batches for such simulation. 
* What result attributes are to be checked? Do they need to be aggregated?